<a href="https://colab.research.google.com/github/nomomon/Texta.ai-Amazon-gpt-neo/blob/main/Texta_ai_Amazon_gpt_neo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare Dataset

In [ ]:
import urllib.request

def download_file(category):
    metadata_url = "http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_"+ category +".json.gz"
    urllib.request.urlretrieve(metadata_url, './data.json.gz')

In [ ]:
import gzip
import json

def read_file():
    counter = 0
    data = []
    with gzip.open('data.json.gz') as f:
        for l in f:
            if counter < 1000:
                data.append(json.loads(l.strip()))
                counter+=1
        
    return data

In [ ]:
download_file("Home_and_Kitchen")

In [ ]:
a = read_file()

In [ ]:
a[0]

In [ ]:
def clean(item):
    if type(item["description"]).__name__ == "str":
        item["description"] = [item["description"]]
    if type(item["category"]).__name__ == "str":
        item["category"] = [item["category"]]
    if type(item["feature"]).__name__ == "str":
        item["feature"] = [item["feature"]]
    return item

In [ ]:
import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [ ]:
def convert_to_prompt(item):
    title = item["title"]
    categories = item["main_cat"] + ", " +", ".join(item["category"])
    short = f"""{item["brand"]}, {item["tech1"]}, {item["tech2"]}"""
    feature = "\n- "+"\n- ".join(item["feature"])
    description = ". ".join(item["description"])

    short = cleanhtml(short)
    description = cleanhtml(description)
    categories = cleanhtml(categories)
    feature = cleanhtml(feature)


    if len(short) > 10 and len(categories) > 5 and len(title) > 5 and len(description) > 10 and len(feature) > 10:
        prompt = f'''
<|endoftext|>Categories: {categories}
Short description: {short}
Title: {title}
Description: {description}
Features: {feature}
<|endoftext|>    
'''
    else:
        prompt = ""

    return prompt

## Put to file

In [ ]:
def put_to_file(category, text):
    f = open(f"datasets/{category}.txt", "a+")
    f.write(text)
    f.close()

## Merge everything

In [ ]:
categories = [
    "All Beauty",
    "Appliances",
    "Arts Crafts and Sewing",
    "Automotive",
    # "Books",
    "CDs and Vinyl",
    "Cell Phones and Accessories",
    # "Digital Music",
    "Electronics",
    # "Gift Cards",
    "Grocery and Gourmet Food",
    "Home and Kitchen",
    "Industrial and Scientific",
    "Kindle Store",
    "Luxury Beauty",
    # "Magazine Subscriptions",
    # "Movies and TV",
    "Musical Instruments",
    "Office Products",
    "Patio Lawn and Garden",
    "Pet Supplies",
    "Prime Pantry",
    # "Software",
    "Sports and Outdoors",
    "Tools and Home Improvement",
    "Toys and Games",
    "Video Games"
]

done = []

In [ ]:
import os, shutil
folder = 'datasets'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
def doAll():
    for category in categories:
        if category not in done:    
            print(f"Doing {category} ...")

            subCat = category.replace(" ", "_")
            download_file(subCat)
            print("     Downloaded")
            for item in read_file():
                put_to_file(subCat, convert_to_prompt(clean(item)))
            print("     Placed in a file")

            done.append(category)

doAll()

## Save the dataset

In [ ]:
import os
import zipfile
    
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), 
                       os.path.relpath(os.path.join(root, file), 
                                       os.path.join(path, '..')))
      
zipf = zipfile.ZipFile('dataset.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('datasets/', zipf)
zipf.close()

In [ ]:
from google.colab import files
files.download('dataset.zip') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# GPT-NEO

In [ ]:
pip install happytransformer

In [ ]:
from happytransformer import HappyGeneration

# Train GPT-Neo

In [ ]:
# happy_gen = HappyGeneration("GPT-NEO", "EleutherAI/gpt-neo-1.3B")
happy_gen = HappyGeneration("GPT-NEO", "EleutherAI/gpt-neo-125M")

Add a text file called "train.txt" to the environment. Also, downgrade to the model to "EleutherAI/gpt-neo-125M" model so that the environment has enough memory    

In [ ]:
from happytransformer import GENTrainArgs

In [ ]:
train_args = GENTrainArgs(learning_rate = 3e-6, num_train_epochs = 1)
happy_gen.train("merge_from_ofoct (1).txt", args = train_args)

08/15/2021 09:10:31 - INFO - happytransformer.happy_transformer -   Preprocessing dataset...


0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-1e781b6cd0fac616/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

08/15/2021 09:10:37 - INFO - happytransformer.happy_transformer -   Training...
PyTorch: setting up devices
***** Running training *****
  Num examples = 708
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 708


Step,Training Loss
500,3.028600


Saving model checkpoint to /tmp/tmpxztc1ahz/checkpoint-500
Configuration saved in /tmp/tmpxztc1ahz/checkpoint-500/config.json
Model weights saved in /tmp/tmpxztc1ahz/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /tmp/tmpxztc1ahz/checkpoint-500/tokenizer_config.json
Special tokens file saved in /tmp/tmpxztc1ahz/checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




In [ ]:
# happy_gen.save("/content/drive/MyDrive/GPT-Neo_Amazon/3/")

# Test Model


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from happytransformer import HappyGeneration

happy_gen = HappyGeneration(load_path="/content/drive/MyDrive/GPT-Neo_Amazon/")

In [ ]:
from happytransformer import GENSettings

In [ ]:
#@title Settings

# https://happytransformer.com/text-generation/settings/

min_length =  10#@param {type:"number"}
max_length = 20 #@param {type:"number"}
do_sample = True #@param {type:"boolean"}
early_stopping = False #@param {type:"boolean"}
num_beams = 1 #@param {type:"number"}
temperature = 0.51 #@param {type:"slider", min:0, max:1, step:0.1}
top_k = 50 #@param {type:"slider", min:1, max:50, step:1}
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
no_repeat_ngram_size = 2 #@param {type:"slider", min:0, max:10, step:1}

gen_args = GENSettings(min_length, max_length, do_sample, early_stopping, num_beams, temperature, top_k, no_repeat_ngram_size, top_p)

In [ ]:
#@title Generate **Title** for Product

category = "Sports and Outdoors"#@param ["All Beauty", "Appliances", "Arts Crafts and Sewing", "Automotive", "CDs and Vinyl", "Cell Phones and Accessories", "Electronics", "Grocery and Gourmet Food", "Home and Kitchen", "Industrial and Scientific", "Kindle Store", "Luxury Beauty", "Musical Instruments", "Office Products", "Patio Lawn and Garden", "Pet Supplies", "Prime Pantry", "Sports and Outdoors", "Tools and Home Improvement", "Toys and Games", "Video Games"]
short = "Fit and Fresh, Cool Coolers Reusable Ice Packs, Set of 4, Clear, Penguins" #@param {type:"string"}
n_examples = 5 #@param {type:"number"}

text = f"""Category: {category}
Short description:{short}
Title: """

output = "\n\n\n"
for i in range(n_examples):
    result = happy_gen.generate_text(text, args=gen_args)

    output += '\n\n>>> ' + result.text.split("Features")[0].split("Description")[0]

print(output)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.







>>>   Ice Pack with Penguins and Penguins

>>>   Ice Pack of 5 - 4 Pack - Clear Penguins - Penguins Icepacks

>>>   Ice Pack - 5 Pack of Icepack Icepacks, Ice pack with Ice packs

>>>   Ice Pack, Icepack Icepacks, 1 Pack

>>>  Ice Packets, Ice pack, reusable ice pack with a pocket, 2 Pack


In [ ]:
#@title Settings

# https://happytransformer.com/text-generation/settings/

min_length =  20#@param {type:"number"}
max_length = 50 #@param {type:"number"}
do_sample = True #@param {type:"boolean"}
early_stopping = False #@param {type:"boolean"}
num_beams = 1 #@param {type:"number"}
temperature = 0.7 #@param {type:"slider", min:0, max:1, step:0.1}
top_k = 50 #@param {type:"slider", min:1, max:50, step:1}
top_p = 1 #@param {type:"slider", min:0, max:1, step:0.1}
no_repeat_ngram_size = 2 #@param {type:"slider", min:0, max:10, step:1}

gen_args = GENSettings(min_length, max_length, do_sample, early_stopping, num_beams, temperature, top_k, no_repeat_ngram_size, top_p)

In [ ]:
#@title Generate **Description** for Product

category = "Sports and Outdoors"#@param ["All Beauty", "Appliances", "Arts Crafts and Sewing", "Automotive", "CDs and Vinyl", "Cell Phones and Accessories", "Electronics", "Grocery and Gourmet Food", "Home and Kitchen", "Industrial and Scientific", "Kindle Store", "Luxury Beauty", "Musical Instruments", "Office Products", "Patio Lawn and Garden", "Pet Supplies", "Prime Pantry", "Sports and Outdoors", "Tools and Home Improvement", "Toys and Games", "Video Games"]
short = "5.25 x 3 x 5.5 inches, 1 pounds, Fit and Fresh, 336KFF, #6 in Sports & Outdoors (See Top 100 in Sports & Outdoors) #1 in Camping Cooler Accessories, , SLIM ICE PACKS, COMPACT DESIGN, LONG-LASTING, HEALTHY AND SAFE" #@param {type:"string"}
title = "Fit & Fresh Cool Coolers Reusable Ice Packs, Set of 4, Clear, Package may vary" #@param {type:"string"}
n_examples = 5 #@param {type:"number"}

text = f"""Category: {category}
Short description: {short}
Title: {title}
Description: """

output = "\n\n\n"
for i in range(n_examples):
    result = happy_gen.generate_text(text, args=gen_args)

    output += '\n\n>>> ' + result.text.split("Features")[0]

print(output)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.







>>>  Fit-Fresh Cooling is a great way to stay cool in your outdoor life and use it on a variety of outdoor activities.  The frozen-cooler is made with a reusable ice pack that can be used as a cool-down item or

>>>  Fit&Fresh Cooling is a lightweight, portable ice pack that fits on most patio decks. It is designed in a compact package with a clean, clean fit and fresh design.  It contains 4 stainless steel bottles, as well as a large metal

>>>  -  Fit Ice Pack  2.0oz. IcePack  4.8oz Icepack  1.6oz ice pack  Package included  

>>>  1. The Fit&Fresh Cooling Reusable Ice Pack  with the Fresh and Clear Package  is easy to clean, durable, and contains enough ice to last you for years  2. Each pack contains 3.6 ounces of ice,

>>>  Fit&Fresh is a cool new and easy-to-use ice pack that's easy to clean, and a great ice scraper that helps you remove ice from your water bottles.  Its perfect for cleaning and having a blast with your family.


In [ ]:
#@title Settings for Features (BulletPoints)

# https://happytransformer.com/text-generation/settings/

min_length =  10#@param {type:"number"}
max_length = 100 #@param {type:"number"}
do_sample = True #@param {type:"boolean"}
early_stopping = True #@param {type:"boolean"}
num_beams = 1 #@param {type:"number"}
temperature = 0.6 #@param {type:"slider", min:0, max:1, step:0.1}
top_k = 50 #@param {type:"slider", min:1, max:50, step:1}
top_p = 0.8 #@param {type:"slider", min:0, max:1, step:0.1}
no_repeat_ngram_size = 1 #@param {type:"slider", min:0, max:10, step:1}

gen_args = GENSettings(min_length, max_length, do_sample, early_stopping, num_beams, temperature, top_k, no_repeat_ngram_size, top_p)

In [ ]:
#@title Generate **Features** for Product

category = "Sports and Outdoors"#@param ["All Beauty", "Appliances", "Arts Crafts and Sewing", "Automotive", "CDs and Vinyl", "Cell Phones and Accessories", "Electronics", "Grocery and Gourmet Food", "Home and Kitchen", "Industrial and Scientific", "Kindle Store", "Luxury Beauty", "Musical Instruments", "Office Products", "Patio Lawn and Garden", "Pet Supplies", "Prime Pantry", "Sports and Outdoors", "Tools and Home Improvement", "Toys and Games", "Video Games"]
short = "5.25 x 3 x 5.5 inches, 1 pounds, Fit and Fresh, 336KFF, #6 in Sports & Outdoors (See Top 100 in Sports & Outdoors) #1 in Camping Cooler Accessories, , SLIM ICE PACKS, COMPACT DESIGN, LONG-LASTING, HEALTHY AND SAFE" #@param {type:"string"}
title = "Fit & Fresh Cool Coolers Reusable Ice Packs, Set of 4, Clear, Package may vary" #@param {type:"string"}
description = "Fit Fresh and freeze, and take up to 70 pounds of ice, so you can stay cool. Use ice packs to cool other ice to the point that it is warmer than ice. The ice pack is manufactured with ice and foam insulation, but it does not contain ice cubes. It contains snow cubes, ice foam, snow cushions and ice gel. Do not use ice-gaugeippers to get ice from the icepack. Ice gel and snow foam are designed to resist any contact with water or ice in the area." #@param {type:"string"}
n_examples =  5#@param {type:"number"}

text = f"""
Category: {category}
Short description: f{short}
Title: {title}
Description: {description}
Features:
- """

output = "\n\n"
for i in range(n_examples):
    result = happy_gen.generate_text(text, args=gen_args)

    output += '\n>>>'
    output += '\n-' + "\n- ".join(result.text.split("- "))
print(output)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.





>>>
- 
- High quality foams for better protection against freezing weather conditions
>>>
- High quality foams for a great fit 
- Foam protects against bumps by preventing them getting into your boat's bottom
>>>
- Easy installation 
- Easily install a fit fresh winter cold hydration kit on your freezer door
>>>
- Perfect for outdoor activities like camping 
- fits into most winter sports equipment
>>>
- Perfect for outdoor camping 
- Includes all four compartments on your car windshield wiper blades; compact design provides a natural fit around most cars' heads


# Как запускать и получать ответы от модели? (для @Rash)

[Видео туториал](https://www.youtube.com/watch?v=GzHJ3NUVtV4)

Все не так уж и сложно. Для начала скачаем нужную библиотеку.

```
!pip install happytransformer
```
Скачиваем модельку с [drive](https://drive.google.com/drive/folders/1a4SclxrGzdjrNlG4sUT3Wzpyn8qqxWLu?usp=sharing). Тут всего 4 варианта (0 тренировалась меньше всех, 3 больше).

Подклучаем эту библиотеку и загружаем выбранную модельку, указывая на нее путь в `directory`.

```
from happytransformer import HappyGeneration

happy_gen = HappyGeneration(load_path="/content/drive/MyDrive/GPT-Neo_Amazon/3/")
```

Далее, можно задать настройки генирации. Арман в этом шарит. Но если все же есть вопросы, что это за настройки, то можно посмотреть их [тут](https://happytransformer.com/text-generation/settings/).
```
min_length =  10
max_length = 100 
do_sample = True
early_stopping = True
num_beams = 1 
temperature = 0.6
top_k = 50
top_p = 0.8
no_repeat_ngram_size = 1

gen_args = GENSettings(min_length, max_length, do_sample, early_stopping, num_beams, temperature, top_k, no_repeat_ngram_size, top_p)

```

Для генерации, запускаем следующию функцию, где `text` это промпт который используем. Примеры таких можно найти в вкладке Test Model этого файла. Результат генерации находится в `result.text`.

```
result = happy_gen.generate_text(text, args=gen_args)

print(result.text)
```

Вроде все. Спасибо за внимание. Было приятно с вами поработать :)